<a href="https://colab.research.google.com/github/cagBRT/Recommendation-Systems/blob/main/neural_collaborative_filter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Recommender systems - Neural Collaborative Filtering
> Demo

- toc: true
- badges: true
- comments: true
- hide: true
- categories: [demo, neural networks, deep learning, recommender systems, paper]
- image: https://raw.githubusercontent.com/murilo-cunha/inteligencia-superficial/master/images/2020-09-11-neural_collaborative_filter/cover.png

Download dependencies and run `tensorboard` in the background:

```python
!pip install tensorflow lightfm pandas
```

```python
%load_ext tensorboard
!tensorboard --logdir 2020-09-11-neural_collaborative_filter/logs &
```

In [ ]:
!pip install lightfm

In [ ]:
# hide
import datetime
import os

import lightfm
import numpy as np
import pandas as pd
import tensorflow as tf
from lightfm import LightFM
from lightfm.datasets import fetch_movielens
from scipy import sparse

In [ ]:
# hide
print(f"Tensorflow version: {tf.__version__}")
print(f"LightFM version: {lightfm.__version__}")
print(f"Pandas version: {pd.__version__}")
print(f"Numpy version: {np.__version__}")

In [ ]:
# hide
TOP_K = 5
N_EPOCHS = 10

# Data

![](https://raw.githubusercontent.com/murilo-cunha/inteligencia-superficial/master/images/2020-09-11-neural_collaborative_filter/matrix_factorization_with_alpha.png "Credit: https://developers.google.com/machine-learning/recommendation/collaborative/basics")

In [ ]:
# hide_input
data = fetch_movielens(min_rating=3.0)


print("Interaction matrix:")
print(data["train"].toarray()[:20, :20])

In [ ]:
# collapse
for dataset in ["test", "train"]:
    data[dataset] = (data[dataset].toarray() > 0).astype("int8")

# Make the ratings binary
print("Interaction matrix:")
print(data["train"][:10, :10])

print("\nRatings:")
unique_ratings = np.unique(data["train"])
print(unique_ratings)

In [ ]:
from typing import List


def wide_to_long(wide: np.array, possible_ratings: List[int]) -> np.array:
    """Go from wide table to long.
    :param wide: wide array with user-item interactions
    :param possible_ratings: list of possible ratings that we may have."""

    def _get_ratings(arr: np.array, rating: int) -> np.array:
        """Generate long array for the rating provided
        :param arr: wide array with user-item interactions
        :param rating: the rating that we are interested"""
        idx = np.where(arr == rating)
        return np.vstack(
            (idx[0], idx[1], np.ones(idx[0].size, dtype="int8") * rating)
        ).T

    long_arrays = []
    for r in possible_ratings:
        long_arrays.append(_get_ratings(wide, r))

    return np.vstack(long_arrays)

In [ ]:
long_train = wide_to_long(data["train"], unique_ratings)
df_train = pd.DataFrame(long_train, columns=["user_id", "item_id", "interaction"])

In [ ]:
# hide_input
print("All interactions:")
df_train.head()

In [ ]:
# hide_input
print("Only positive interactions:")
df_train[df_train["interaction"] > 0].head()

# The model (Neural Collaborative Filtering)

<center><img src="https://raw.githubusercontent.com/murilo-cunha/inteligencia-superficial/master/images/2020-09-11-neural_collaborative_filter/ncf_all_with_alpha.png" width="70%" url="https://developers.google.com/machine-learning/recommendation/collaborative/basics" description="Fonte: https://developers.google.com/machine-learning/recommendation/collaborative/basics" /> </center>

In [ ]:
import keras as keras
from keras.layers import (
    Concatenate,
    Dense,
    Embedding,
    Flatten,
    Input,
    Multiply,
)
from keras.models import Model
from keras.regularizers import l2


def create_ncf(
    number_of_users: int,
    number_of_items: int,
    latent_dim_mf: int = 4,
    latent_dim_mlp: int = 32,
    reg_mf: int = 0,
    reg_mlp: int = 0.01,
    dense_layers: List[int] = [8, 4],
    reg_layers: List[int] = [0.01, 0.01],
    activation_dense: str = "relu",
) -> keras.Model:

    # input layer
    user = Input(shape=(), dtype="int32", name="user_id")
    item = Input(shape=(), dtype="int32", name="item_id")

    # embedding layers
    mf_user_embedding = Embedding(
        input_dim=number_of_users,
        output_dim=latent_dim_mf,
        name="mf_user_embedding",
        embeddings_initializer="RandomNormal",
        embeddings_regularizer=l2(reg_mf),
        input_length=1,
    )
    mf_item_embedding = Embedding(
        input_dim=number_of_items,
        output_dim=latent_dim_mf,
        name="mf_item_embedding",
        embeddings_initializer="RandomNormal",
        embeddings_regularizer=l2(reg_mf),
        input_length=1,
    )

    mlp_user_embedding = Embedding(
        input_dim=number_of_users,
        output_dim=latent_dim_mlp,
        name="mlp_user_embedding",
        embeddings_initializer="RandomNormal",
        embeddings_regularizer=l2(reg_mlp),
        input_length=1,
    )
    mlp_item_embedding = Embedding(
        input_dim=number_of_items,
        output_dim=latent_dim_mlp,
        name="mlp_item_embedding",
        embeddings_initializer="RandomNormal",
        embeddings_regularizer=l2(reg_mlp),
        input_length=1,
    )

    # MF vector
    mf_user_latent = Flatten()(mf_user_embedding(user))
    mf_item_latent = Flatten()(mf_item_embedding(item))
    mf_cat_latent = Multiply()([mf_user_latent, mf_item_latent])

    # MLP vector
    mlp_user_latent = Flatten()(mlp_user_embedding(user))
    mlp_item_latent = Flatten()(mlp_item_embedding(item))
    mlp_cat_latent = Concatenate()([mlp_user_latent, mlp_item_latent])

    mlp_vector = mlp_cat_latent

    # build dense layers for model
    for i in range(len(dense_layers)):
        layer = Dense(
            dense_layers[i],
            activity_regularizer=l2(reg_layers[i]),
            activation=activation_dense,
            name="layer%d" % i,
        )
        mlp_vector = layer(mlp_vector)

    predict_layer = Concatenate()([mf_cat_latent, mlp_vector])

    result = Dense(
        1, activation="sigmoid", kernel_initializer="lecun_uniform", name="interaction"
    )

    output = result(predict_layer)

    model = Model(
        inputs=[user, item],
        outputs=[output],
    )

    return model

In [ ]:
# collapse
from keras.optimizers import Adam

n_users, n_items = data["train"].shape
ncf_model = create_ncf(n_users, n_items)

ncf_model.compile(
    optimizer=Adam(),
    loss="binary_crossentropy",
    metrics=[
        tf.keras.metrics.TruePositives(name="tp"),
        tf.keras.metrics.FalsePositives(name="fp"),
        tf.keras.metrics.TrueNegatives(name="tn"),
        tf.keras.metrics.FalseNegatives(name="fn"),
        tf.keras.metrics.BinaryAccuracy(name="accuracy"),
        tf.keras.metrics.Precision(name="precision"),
        tf.keras.metrics.Recall(name="recall"),
        tf.keras.metrics.AUC(name="auc"),
    ],
)
ncf_model._name = "neural_collaborative_filtering"
ncf_model.summary()

In [ ]:
def make_tf_dataset(
    df: pd.DataFrame,
    targets: List[str],
    val_split: float = 0.1,
    batch_size: int = 512,
    seed=42,
):
    """Make TensorFlow dataset from Pandas DataFrame.
    :param df: input DataFrame - only contains features and target(s)
    :param targets: list of columns names corresponding to targets
    :param val_split: fraction of the data that should be used for validation
    :param batch_size: batch size for training
    :param seed: random seed for shuffling data - `None` won't shuffle the data"""

    n_val = round(df.shape[0] * val_split)
    if seed:
        # shuffle all the rows
        x = df.sample(frac=1, random_state=seed).to_dict("series")
    else:
        x = df.to_dict("series")
    y = dict()
    for t in targets:
        y[t] = x.pop(t)
    ds = tf.data.Dataset.from_tensor_slices((x, y))

    ds_val = ds.take(n_val).batch(batch_size)
    ds_train = ds.skip(n_val).batch(batch_size)
    return ds_train, ds_val

In [ ]:
# create train and validation datasets
ds_train, ds_val = make_tf_dataset(df_train, ["interaction"])

In [33]:
%%time
# define logs and callbacks
logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)
early_stopping_callback = tf.keras.callbacks.EarlyStopping(
    monitor="val_loss", patience=0
)

train_hist = ncf_model.fit(
    ds_train,
    validation_data=ds_val,
    epochs=N_EPOCHS,
    callbacks=[tensorboard_callback, early_stopping_callback],
    verbose=1,
)

2789/2789 [==============================] - 25s 9ms/step - loss: 0.1108 - tp: 16185.0000 - fp: 8140.0000 - tn: 1352274.0000 - fn: 50914.0000 - accuracy: 0.9586 - precision: 0.6654 - recall: 0.2412 - auc: 0.9344 - val_loss: 0.1131 - val_tp: 1868.0000 - val_fp: 922.0000 - val_tn: 150163.0000 - val_fn: 5660.0000 - val_accuracy: 0.9585 - val_precision: 0.6695 - val_recall: 0.2481 - val_auc: 0.9303
Epoch 6/10
2789/2789 [==============================] - 19s 7ms/step - loss: 0.1088 - tp: 17377.0000 - fp: 8793.0000 - tn: 1351621.0000 - fn: 49722.0000 - accuracy: 0.9590 - precision: 0.6640 - recall: 0.2590 - auc: 0.9375 - val_loss: 0.1120 - val_tp: 1973.0000 - val_fp: 980.0000 - val_tn: 150105.0000 - val_fn: 5555.0000 - val_accuracy: 0.9588 - val_precision: 0.6681 - val_recall: 0.2621 - val_auc: 0.9321
Epoch 7/10
2789/2789 [==============================] - 19s 7ms/step - loss: 0.1077 - tp: 18026.0000 - fp: 9166.0000 - tn: 1351248.0000 - fn: 49073.0000 - accuracy: 0.9592 - precision: 0.6629 -

In [34]:
long_test = wide_to_long(data["train"], unique_ratings)
df_test = pd.DataFrame(long_test, columns=["user_id", "item_id", "interaction"])
ds_test, _ = make_tf_dataset(df_test, ["interaction"], val_split=0, seed=None)

In [35]:
%%time
ncf_predictions = ncf_model.predict(ds_test)
df_test["ncf_predictions"] = ncf_predictions

3098/3098 [==============================] - 9s 3ms/step
CPU times: user 13.6 s, sys: 382 ms, total: 14 s
Wall time: 10.4 s


In [36]:
# hide_input
df_test.head()

,user_id,item_id,interaction,ncf_predictions
0,0,7,0,0.410340
1,0,10,0,0.617093
2,0,19,0,0.042952
3,0,20,0,0.129163
4,0,26,0,0.202167


In [37]:
# hide
# sanity checks
# stop execution if low standard deviation (all recommendations are the same)
std = df_test.describe().loc["std", "ncf_predictions"]
if std < 0.01:
    raise ValueError("Model predictions have standard deviation of less than 1e-2.")

In [38]:
# collapse
data["ncf_predictions"] = df_test.pivot(
    index="user_id", columns="item_id", values="ncf_predictions"
).values
print("Neural collaborative filtering predictions")
print(data["ncf_predictions"][:10, :4])

Neural collaborative filtering predictions
[[5.8425564e-01 4.9049383e-01 2.2775204e-01 7.2633284e-01]
 [3.5103866e-01 4.0277583e-03 7.1151541e-03 2.3901581e-03]
 [6.8950718e-03 2.2802419e-04 3.6096867e-02 2.7788676e-05]
 [2.3498936e-02 6.6564552e-04 6.4764405e-03 4.3370528e-04]
 [4.4726932e-01 2.6358682e-01 1.5195049e-01 4.5011023e-01]
 [2.3893301e-01 5.9842374e-02 8.9173065e-03 3.1924850e-01]
 [6.6076672e-01 6.0416049e-01 1.4748059e-01 8.7005430e-01]
 [5.1572829e-01 6.2441353e-02 2.3768341e-02 1.4747883e-01]
 [1.0881488e-01 1.0730972e-03 7.4909086e-04 3.6570290e-03]
 [2.9066503e-01 1.1760246e-01 2.4980199e-02 4.2411038e-01]]


In [39]:
precision_ncf = tf.keras.metrics.Precision(top_k=TOP_K)
recall_ncf = tf.keras.metrics.Recall(top_k=TOP_K)

precision_ncf.update_state(data["test"], data["ncf_predictions"])
recall_ncf.update_state(data["test"], data["ncf_predictions"])
print(
    f"At K = {TOP_K}, we have a precision of {precision_ncf.result().numpy():.5f}",
    "and a recall of {recall_ncf.result().numpy():.5f}",
)

At K = 5, we have a precision of 0.10138 and a recall of {recall_ncf.result().numpy():.5f}


In [40]:
%%time
# LightFM model
def norm(x: float) -> float:
    """Normalize vector"""
    return (x - np.min(x)) / np.ptp(x)


lightfm_model = LightFM(loss="warp")
lightfm_model.fit(sparse.coo_matrix(data["train"]), epochs=N_EPOCHS)

lightfm_predictions = lightfm_model.predict(
    df_test["user_id"].values, df_test["item_id"].values
)
df_test["lightfm_predictions"] = lightfm_predictions
wide_predictions = df_test.pivot(
    index="user_id", columns="item_id", values="lightfm_predictions"
).values
data["lightfm_predictions"] = norm(wide_predictions)

# compute the metrics
precision_lightfm = tf.keras.metrics.Precision(top_k=TOP_K)
recall_lightfm = tf.keras.metrics.Recall(top_k=TOP_K)
precision_lightfm.update_state(data["test"], data["lightfm_predictions"])
recall_lightfm.update_state(data["test"], data["lightfm_predictions"])
print(
    f"At K = {TOP_K}, we have a precision of {precision_lightfm.result().numpy():.5f}",
    "and a recall of {recall_lightfm.result().numpy():.5f}",
)

At K = 5, we have a precision of 0.11113 and a recall of {recall_lightfm.result().numpy():.5f}
CPU times: user 1.18 s, sys: 74.1 ms, total: 1.26 s
Wall time: 1.2 s
